In [1]:
include("import_all.jl")

build_min_heap! (generic function with 1 method)

In [2]:
# io = open("log.txt", "w+")
# logger = SimpleLogger(io)
logger = ConsoleLogger(stdout, Logging.Info)
# fileLogger = SimpleLogger(io, Logging.Debug)
# global_logger(fileLogger)
global_logger(logger)

ConsoleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(Base.Libc.WindowsRawSocket(0x00000000000003cc) open, 0 bytes waiting))), Info, Logging.default_metafmt, true, 0, Dict{Any, Int64}())

In [4]:
using Profile
using PProf

In [5]:
function wrapper()
    max_time = 1440
    start_time = 345

    train_period = 2
    train_capacity = 1000

    @info "$(now()): initialization starting at time "
    station_dict = construct_station_dict("data/input/station_data.csv")

    station_name_id_map = construct_station_name_id_map(station_dict)

    # construct the edges
    start_stations = construct_edges_from_edges_dict!(station_dict, ["tel", "ccl", "ewl", "nsl", "nel", "cgl", "dtl"])

    lines = construct_lines_from_start_stations(station_dict, start_stations)

    commuter_graph = construct_commuter_graph(station_dict)

    floyd_warshall!(commuter_graph)

    get_all_path_pairs!(commuter_graph)

    paths = get_interchange_paths(station_dict, lines, commuter_graph)

    trains = Dict()
    event_queue = PriorityQueue{Event, Float64}()

    for line_code in keys(lines)
        line_duration = get_line_duration(station_dict, lines, line_code)
        depot_id = lines[line_code]["FW"][1]
        result = create_period_train_placement_events(line_code, line_duration, train_period, train_capacity, depot_id, "FW", start_time)

        for (k,v) in result["trains"]
            trains[k] = v 
        end 

        for i in result["events"]
            enqueue!(event_queue, i, i.time)
        end
    end

    spawn_events = create_spawn_events!("data/input/spawn_data.csv", station_dict, start_time)

    for i in spawn_events 
        enqueue!(event_queue, i, i.time)
    end 

    metro = Metro(station_dict, trains, lines, paths);


    data_store = Data_Store(Dict(), Dict(), Dict(), Dict(), Dict())
    @info "$(now()): initialization finish "

    @info "$(now()): starting simulation "
    final_data = simulate!(max_time, metro, event_queue, data_store)
    @info "$(now()): ending simulation "

    @info "storing data"
    store_final_data(final_data, max_time)
end


wrapper (generic function with 1 method)

In [6]:
wrapper()

[ Info: 2023-04-19T17:24:13.546: initialization starting at time 
[ Info: 2023-04-19T17:24:27.296: initialization finish 
[ Info: 2023-04-19T17:24:27.328: starting simulation 
[ Info: 2023-04-19T17:24:27.359: start time is 5
[ Info: processed 5293 events
[ Info: 2023-04-19T17:24:30.230: time is now 6
[ Info: size of event queue 17146
[ Info: processed 100537 events
[ Info: 2023-04-19T17:24:31.832: time is now 7
[ Info: size of event queue 17128
[ Info: processed 262436 events
[ Info: 2023-04-19T17:24:35.057: time is now 8
[ Info: size of event queue 17033
[ Info: processed 407565 events
[ Info: 2023-04-19T17:24:38.742: time is now 9
[ Info: size of event queue 16944
[ Info: processed 505761 events
[ Info: 2023-04-19T17:24:41.260: time is now 10
[ Info: size of event queue 16864
[ Info: processed 589251 events
[ Info: 2023-04-19T17:24:43.096: time is now 11
[ Info: size of event queue 16752
[ Info: processed 676027 events
[ Info: 2023-04-19T17:24:44.962: time is now 12
[ Info: size of e

In [7]:
using BenchmarkTools

In [8]:
@profile wrapper()

[ Info: 2023-04-19T17:25:39.005: initialization starting at time 
[ Info: 2023-04-19T17:25:43.519: initialization finish 
[ Info: 2023-04-19T17:25:43.551: starting simulation 
[ Info: 2023-04-19T17:25:43.574: start time is 5
[ Info: processed 5295 events
[ Info: 2023-04-19T17:25:43.640: time is now 6
[ Info: size of event queue 17167
[ Info: processed 100500 events
[ Info: 2023-04-19T17:25:45.340: time is now 7
[ Info: size of event queue 17163
[ Info: processed 260647 events
[ Info: 2023-04-19T17:25:48.573: time is now 8
[ Info: size of event queue 17087
[ Info: processed 405369 events
[ Info: 2023-04-19T17:25:51.975: time is now 9
[ Info: size of event queue 16983
[ Info: processed 502880 events
[ Info: 2023-04-19T17:25:54.524: time is now 10
[ Info: size of event queue 16872
[ Info: processed 586453 events
[ Info: 2023-04-19T17:25:56.320: time is now 11
[ Info: size of event queue 16760
[ Info: processed 673867 events
[ Info: 2023-04-19T17:25:58.220: time is now 12
[ Info: size of e

In [9]:
pprof(;webport=58698)

"profile.pb.gz"

In [10]:
Profile.print(format=:tree, recur=:flat)

Overhead ╎ [+additional indent] Count File:Line; Function
   ╎3232 @Base\task.jl:484; (::IJulia.var"#15#18")()
   ╎ 3232 ...lia\src\eventloop.jl:8; eventloop(socket::ZMQ.Socket)
   ╎  3232 @Base\essentials.jl:726; invokelatest
   ╎   3232 @Base\essentials.jl:729; #invokelatest#2
   ╎    3232 ...\execute_request.jl:67; execute_request(socket::ZMQ.So...
   ╎     3232 ...SoftGlobalScope.jl:65; softscope_include_string(m::M...
   ╎    ╎ 3232 @Base\loading.jl:1428; include_string(mapexpr::typ...
   ╎    ╎  3232 @Base\boot.jl:368; eval
   ╎    ╎   1    In[5]:14; wrapper()
   ╎    ╎    1    ...tion_functions.jl:52; construct_edges_from_edges_...
   ╎    ╎     1    ...ion_functions.jl:87; construct_edges_for_line!(...
   ╎    ╎    ╎ 1    @Base\dict.jl:497; getindex(h::Dict{Any, Any...
   ╎    ╎    ╎  1    @Base\dict.jl:291; ht_keyindex(h::Dict{Any,...
   ╎    ╎    ╎   1    @Base\array.jl:924; getindex
   ╎    ╎   158  In[5]:20; wrapper()
 27╎    ╎    59   ...ding_functions.jl:0; floyd_warshall

Main binary filename not available.
Serving web UI on http://localhost:58698


  1╎    ╎    ╎  1    @Base\array.jl:966; setindex!
   ╎    ╎    1    ...ing_functions.jl:109; get_all_path_pairs!(commut...
   ╎    ╎     1    @Base\dict.jl:497; getindex(h::Dict{String, ...
   ╎    ╎    ╎ 1    @Base\dict.jl:284; ht_keyindex(h::Dict{Strin...
   ╎    ╎    ╎  1    @Base\dict.jl:169; hashindex
   ╎    ╎    ╎   1    @Base\hashing.jl:20; hash
  1╎    ╎    ╎    1    @Base\hashing.jl:107; hash
  1╎    ╎    1    ...ing_functions.jl:110; get_all_path_pairs!(commut...
   ╎    ╎    1    ...ing_functions.jl:113; get_all_path_pairs!(commut...
   ╎    ╎     1    @Base\dict.jl:383; setindex!(h::Dict{String,...
   ╎    ╎    ╎ 1    @Base\dict.jl:309; ht_keyindex2!(h::Dict{Str...
   ╎    ╎    ╎  1    @Base\dict.jl:169; hashindex
   ╎    ╎    ╎   1    @Base\hashing.jl:20; hash
  1╎    ╎    ╎    1    @Base\hashing.jl:107; hash
   ╎    ╎   17   In[5]:24; wrapper()
  1╎    ╎    1    ...ing_functions.jl:132; get_interchange_paths(stat...
  1╎    ╎    2    ...ing_functions.jl:136; get_interch

  1╎    ╎     6    @Base\broadcast.jl:1298; broadcasted
  5╎    ╎    ╎ 5    @Base\broadcast.jl:1306; broadcasted
   ╎    ╎     3    @Base\broadcast.jl:860; materialize(bc::Base.Broa...
   ╎    ╎    ╎ 3    @Base\broadcast.jl:885; copy
   ╎    ╎    ╎  3    @Base\broadcast.jl:913; copyto!
   ╎    ╎    ╎   3    @Base\broadcast.jl:960; copyto!
   ╎    ╎    ╎    3    @Base\simdloop.jl:77; macro expansion
   ╎    ╎    ╎     3    ...e\broadcast.jl:961; macro expansion
   ╎    ╎    ╎    ╎ 3    ...\broadcast.jl:597; getindex
   ╎    ╎    ╎    ╎  3    ...\broadcast.jl:643; _broadcast_getindex
   ╎    ╎    ╎    ╎   3    ...\broadcast.jl:670; _broadcast_getindex_e...
   ╎    ╎    ╎    ╎    3    ...\substring.jl:65; String
  3╎    ╎    ╎    ╎     3    ...gs\string.jl:69; unsafe_string
   ╎    ╎     6    ...strings\util.jl:592; split(str::String, splitt...
   ╎    ╎    ╎ 6    ...strings\util.jl:595; #split#438
   ╎    ╎    ╎  6    @Base\array.jl:647; collect
   ╎    ╎    ╎   2    @Base\array.jl:652; 

   ╎    ╎    ╎     1    @Base\io.jl:419; unsafe_write
   ╎    ╎    ╎    ╎ 1    @Base\stream.jl:1118; unsafe_write(s::Base....
   ╎    ╎    ╎    ╎  1    @Base\stream.jl:1046; uv_write(s::Base.Pipe...
   ╎    ╎    ╎    ╎   1    @Base\task.jl:932; wait()
  1╎    ╎    ╎    ╎    1    @Base\libuv.jl:104; process_events
   ╎    ╎    1    ...mul_functions.jl:203; simulate!(max_time::Int64,...
  1╎    ╎     1    @Base\logging.jl:365; macro expansion
 21╎    ╎    1761 ...mul_functions.jl:208; simulate!(max_time::Int64,...
  5╎    ╎     5    @Base\boot.jl:370; kwfunc(f::Any)
  3╎    ╎     3    @Base\namedtuple.jl:299; merge(a::NamedTuple{(), T...
   ╎    ╎     6    @Base\namedtuple.jl:300; merge(a::NamedTuple{(), T...
   ╎    ╎    ╎ 6    @Base\array.jl:404; getindex
  6╎    ╎    ╎  6    @Base\boot.jl:459; Array
   ╎    ╎     3    @Base\namedtuple.jl:301; merge(a::NamedTuple{(), T...
   ╎    ╎    ╎ 3    @Base\array.jl:426; getindex
   ╎    ╎    ╎  3    @Base\boot.jl:478; Array
  3╎    ╎    ╎   3  

...

   ╎    ╎    ╎    ╎ 5    @Base\env.jl:80; get
   ╎    ╎    ╎    ╎  3    @Base\env.jl:11; access_env(onError::Re...
   ╎    ╎    ╎    ╎   3    @Base\c.jl:250; cwstring(s::String)
   ╎    ╎    ╎    ╎    1    @Base\c.jl:311; transcode(#unused#::...
   ╎    ╎    ╎    ╎     1    @Base\array.jl:404; getindex
  1╎    ╎    ╎    ╎    ╎ 1    @Base\boot.jl:459; Array
   ╎    ╎    ╎    ╎    1    @Base\c.jl:314; transcode(#unused#::...
  1╎    ╎    ╎    ╎     1    @Base\array.jl:1267; sizehint!
   ╎    ╎    ╎    ╎    1    @Base\c.jl:352; transcode(#unused#::...
   ╎    ╎    ╎    ╎     1    @Base\array.jl:1058; push!
  1╎    ╎    ╎    ╎    ╎ 1    @Base\array.jl:1011; _growend!
   ╎    ╎    ╎    ╎  2    @Base\env.jl:12; access_env(onError::Re...
  2╎    ╎    ╎    ╎   2    @Base\env.jl:6; _getenvlen
   ╎    ╎    ╎  18   ...on_functions.jl:170; alight_commuters!(time::...
   ╎    ╎    ╎   18   @Base\dict.jl:125; Dict
  1╎    ╎    ╎    1    @Base\dict.jl:128; Dict(kv::Tuple{Pair{Str...
  1╎    ╎    ╎   

   ╎    ╎    ╎    6    @Base\boot.jl:478; Array
  6╎    ╎    ╎     6    @Base\boot.jl:459; Array
   ╎    ╎    ╎  7    ...me\dataframe.jl:311; DataFrame(; kwargs::Base...
   ╎    ╎    ╎   7    @Base\iterators.jl:259; iterate
   ╎    ╎    ╎    7    ...e\iterators.jl:252; _pairs_elt
  7╎    ╎    ╎     7    ...\namedtuple.jl:137; getindex
   ╎    ╎    ╎  5    ...me\dataframe.jl:322; DataFrame(; kwargs::Base...
   ╎    ╎    ╎   4    @Base\array.jl:1058; push!
  4╎    ╎    ╎    4    @Base\array.jl:1011; _growend!
   ╎    ╎    ╎   1    @Base\array.jl:1059; push!
  1╎    ╎    ╎    1    @Base\array.jl:966; setindex!
   ╎    ╎    ╎  10   ...me\dataframe.jl:323; DataFrame(; kwargs::Base...
   ╎    ╎    ╎   9    @Base\array.jl:1064; push!
  9╎    ╎    ╎    9    @Base\array.jl:1011; _growend!
  1╎    ╎    ╎   1    @Base\array.jl:1065; push!
   ╎    ╎    ╎  5    ...me\dataframe.jl:325; DataFrame(; kwargs::Base...
   ╎    ╎    ╎   5    @Base\iterators.jl:259; iterate
   ╎    ╎    ╎    5    ...e\itera

  1╎    ╎    ╎    ╎     1    @Base\dict.jl:683; delete!(h::Dict{Sym...
   ╎    ╎    ╎    ╎    1    @Base\set.jl:66; in
   ╎    ╎    ╎    ╎     1    @Base\dict.jl:569; haskey
  1╎    ╎    ╎    ╎    ╎ 1    @Base\dict.jl:288; ht_keyindex
   ╎    ╎    ╎    ╎    1    @Base\set.jl:67; push!
   ╎    ╎    ╎    ╎     1    @Base\dict.jl:390; setindex!(h::Dict{S...
   ╎    ╎    ╎    ╎    ╎ 1    @Base\dict.jl:366; _setindex!
   ╎    ╎    ╎    ╎    ╎  1    @Base\int.jl:501; >>
  1╎    ╎    ╎    ╎    ╎   1    @Base\int.jl:494; >>
   ╎    ╎    ╎    ╎  6    @Base\array.jl:2664; _grow!
  2╎    ╎    ╎    ╎   2    ...bstractset.jl:499; mapfilter(pred::Base....
   ╎    ╎    ╎    ╎   2    ...bstractset.jl:501; mapfilter(pred::Base....
  1╎    ╎    ╎    ╎    1    @Base\array.jl:2649; (::Base.var"#132#13...
   ╎    ╎    ╎    ╎    1    @Base\array.jl:2650; (::Base.var"#132#13...
   ╎    ╎    ╎    ╎     1    @Base\set.jl:66; in
   ╎    ╎    ╎    ╎    ╎ 1    @Base\dict.jl:569; haskey
  1╎    ╎    ╎    ╎    ╎  1

   ╎    ╎    ╎  1    @Base\dict.jl:283; ht_keyindex(h::Dict{Stri...
  1╎    ╎    ╎   1    @Base\Base.jl:38; getproperty
   ╎    ╎     5    ...ore_functions.jl:80; update_wait_time!(data_sto...
   ╎    ╎    ╎ 2    @Base\array.jl:1108; append!
  2╎    ╎    ╎  2    @Base\array.jl:1011; _growend!
   ╎    ╎    ╎ 3    @Base\array.jl:1109; append!
  1╎    ╎    ╎  3    @Base\array.jl:322; copyto!
   ╎    ╎    ╎   2    @Base\array.jl:331; _copyto_impl!(dest::Vect...
  2╎    ╎    ╎    2    @Base\array.jl:289; unsafe_copyto!
 10╎    ╎    12   ...mul_functions.jl:215; simulate!(max_time::Int64,...
  1╎    ╎     2    @Base\dict.jl:707; iterate(t::Dict{String, A...
   ╎    ╎    ╎ 1    @Base\dict.jl:687; skip_deleted
  1╎    ╎    ╎  1    @Base\Base.jl:38; getproperty
 10╎    ╎    27   ...mul_functions.jl:219; simulate!(max_time::Int64,...
   ╎    ╎     8    @Base\array.jl:125; vect
   ╎    ╎    ╎ 8    @Base\boot.jl:478; Array
  8╎    ╎    ╎  8    @Base\boot.jl:459; Array
  1╎    ╎     1    @Base\dict

   ╎    ╎    ╎   30   ...src\datasets.jl:305; #write#57
   ╎    ╎    ╎    30   ...rc\datasets.jl:278; write_dataset
   ╎    ╎    ╎     2    ...rc\datasets.jl:281; write_dataset(parent::...
   ╎    ╎    ╎    ╎ 2    ...c\datasets.jl:264; create_dataset
   ╎    ╎    ╎    ╎  1    ...c\datasets.jl:268; create_dataset(parent...
   ╎    ╎    ╎    ╎   1    ...dataspaces.jl:71; dataspace
   ╎    ╎    ╎    ╎    1    ...dataspaces.jl:71; #dataspace#38
   ╎    ╎    ╎    ╎     1    ...ataspaces.jl:69; _dataspace
  1╎    ╎    ╎    ╎    ╎ 1    ...unctions.jl:6474; h5s_create_simple(...
   ╎    ╎    ╎    ╎  1    ...c\datasets.jl:270; create_dataset(parent...
   ╎    ╎    ╎    ╎   1    ...c\datasets.jl:40; create_dataset(parent...
   ╎    ╎    ╎    ╎    1    ...c\datasets.jl:60; create_dataset(parent...
  1╎    ╎    ╎    ╎     1    ...functions.jl:470; h5d_create(loc_id::...
   ╎    ╎    ╎     28   ...rc\datasets.jl:283; write_dataset(parent::...
   ╎    ╎    ╎    ╎ 28   ...c\datasets.jl:493; write_dat

In [11]:
@benchmark wrapper()

[ Info: 2023-04-19T17:27:19.968: initialization starting at time 
[ Info: 2023-04-19T17:27:24.683: initialization finish 
[ Info: 2023-04-19T17:27:24.683: starting simulation 
[ Info: 2023-04-19T17:27:24.683: start time is 5
[ Info: processed 5225 events
[ Info: 2023-04-19T17:27:24.762: time is now 6
[ Info: size of event queue 17155
[ Info: processed 100677 events
[ Info: 2023-04-19T17:27:26.473: time is now 7
[ Info: size of event queue 17141
[ Info: processed 261872 events
[ Info: 2023-04-19T17:27:30.338: time is now 8
[ Info: size of event queue 17038
[ Info: processed 407047 events
[ Info: 2023-04-19T17:27:33.849: time is now 9
[ Info: size of event queue 16946
[ Info: processed 504828 events
[ Info: 2023-04-19T17:27:36.333: time is now 10
[ Info: size of event queue 16866
[ Info: processed 588279 events
[ Info: 2023-04-19T17:27:38.195: time is now 11
[ Info: size of event queue 16764
[ Info: processed 674949 events
[ Info: 2023-04-19T17:27:40.218: time is now 12
[ Info: size of e

[ Info: size of event queue 13514
[ Info: processed 1675541 events
[ Info: 2023-04-19T17:28:50.819: time is now 20
[ Info: size of event queue 12144
[ Info: processed 1770562 events
[ Info: 2023-04-19T17:28:52.787: time is now 21
[ Info: size of event queue 10709
[ Info: processed 1857702 events
[ Info: 2023-04-19T17:28:54.604: time is now 22
[ Info: size of event queue 8404
[ Info: processed 1924341 events
[ Info: 2023-04-19T17:28:56.082: time is now 23
[ Info: size of event queue 4802
[ Info: processed 1963586 events
[ Info: 2023-04-19T17:28:57.096: time is now 24
[ Info: size of event queue 591
[ Info: 2023-04-19T17:28:57.096: ending simulation 
[ Info: storing data
[ Info: 2023-04-19T17:28:59.672: initialization starting at time 
[ Info: 2023-04-19T17:29:04.323: initialization finish 
[ Info: 2023-04-19T17:29:04.354: starting simulation 
[ Info: 2023-04-19T17:29:04.354: start time is 5
[ Info: processed 5192 events
[ Info: 2023-04-19T17:29:04.417: time is now 6
[ Info: size of even

[ Info: size of event queue 16529
[ Info: processed 867210 events
[ Info: 2023-04-19T17:30:12.914: time is now 14
[ Info: size of event queue 16399
[ Info: processed 965668 events
[ Info: 2023-04-19T17:30:14.979: time is now 15
[ Info: size of event queue 16204
[ Info: processed 1070400 events
[ Info: 2023-04-19T17:30:17.186: time is now 16
[ Info: size of event queue 15975
[ Info: processed 1185965 events
[ Info: 2023-04-19T17:30:19.521: time is now 17
[ Info: size of event queue 15666
[ Info: processed 1356504 events
[ Info: 2023-04-19T17:30:22.744: time is now 18
[ Info: size of event queue 14965
[ Info: processed 1552537 events
[ Info: 2023-04-19T17:30:26.418: time is now 19
[ Info: size of event queue 13490
[ Info: processed 1676471 events
[ Info: 2023-04-19T17:30:28.980: time is now 20
[ Info: size of event queue 12054
[ Info: processed 1771084 events
[ Info: 2023-04-19T17:30:30.981: time is now 21
[ Info: size of event queue 10599
[ Info: processed 1858368 events
[ Info: 2023-04

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 48.366 s (7.61% GC) to evaluate,
 with a memory estimate of 21.83 GiB, over 380652804 allocations.

In [ ]:
@time wrapper()